# Week 7: Advanced Volatility Modeling

---

## 🎯 What You'll Learn This Week

Volatility is the heartbeat of financial markets. Understanding it is essential for:
- **Risk managers** calculating potential losses
- **Options traders** pricing derivatives
- **Quants** building trading strategies

**By the end of this week, you'll understand:**
- Why volatility "clusters" (big moves follow big moves)
- How to forecast tomorrow's volatility using GARCH models
- The "leverage effect" - why stocks fall faster than they rise
- How to model volatility across multiple assets

**Real-World Impact:**
The 2008 financial crisis showed what happens when volatility models fail. VaR models using constant volatility dramatically underestimated risk. Modern quants MUST understand volatility dynamics!

---

## Table of Contents
1. Volatility Stylized Facts
2. GARCH Model
3. EGARCH (Exponential GARCH)
4. GJR-GARCH (Asymmetric Effects)
5. Multivariate GARCH (DCC)

---

## 1. Volatility Stylized Facts

### 🤔 Why Model Volatility? (Simple Explanation)

Think of volatility as "how wild the market is acting today."

**Low volatility** = calm waters, stock moves 0.5% daily
**High volatility** = stormy seas, stock moves 3-5% daily

**Who cares about volatility?**
- **Risk managers**: "How much could we lose tomorrow?" (VaR)
- **Options traders**: Volatility literally IS what you're trading!
- **Portfolio managers**: Need to know which assets are risky
- **Interviewers**: This is a TOP quant interview topic!

### The 4 "Laws" of Volatility (Stylized Facts)

**1. Volatility Clustering** 📊
> "Calm days follow calm days, crazy days follow crazy days"

When markets crash, they stay volatile for weeks. When markets are quiet, they stay quiet. This is WHY we need models like GARCH!

**2. Mean Reversion** ↩️
> "Volatility always comes back to normal eventually"

Even after a crisis (2008, COVID 2020), volatility eventually calms down. Markets don't stay panicked forever.

**3. Leverage Effect (Asymmetry)** ⚠️
> "Stocks take the stairs up, but the elevator down"

A -3% day increases volatility MORE than a +3% day. Bad news = more fear = more volatility. This is why EGARCH and GJR-GARCH exist!

**4. Fat Tails** 🦏
> "Crashes happen more often than math predicts"

The normal distribution says a -10% day is nearly impossible. But we've seen many! This is why we use t-distributions and stress testing.

In [6]:
import numpy as np
import pandas as pd
from scipy import stats

np.random.seed(42)

# Simulate returns exhibiting volatility clustering
n = 1000
returns = np.zeros(n)
volatility = np.zeros(n)
volatility[0] = 0.01

# GARCH-like process
omega, alpha, beta = 0.00001, 0.1, 0.85
for t in range(1, n):
    volatility[t] = np.sqrt(omega + alpha * returns[t-1]**2 + beta * volatility[t-1]**2)
    returns[t] = volatility[t] * np.random.normal()

# Test for volatility clustering: autocorrelation of squared returns
from statsmodels.tsa.stattools import acf
acf_squared = acf(returns**2, nlags=10)

print("Volatility Stylized Facts")
print("="*50)

# 1. Volatility clustering
print("\n1. VOLATILITY CLUSTERING")
print("   Autocorrelation of squared returns (r²):")
for lag in [1, 5, 10]:
    print(f"   Lag {lag}: {acf_squared[lag]:.3f}")
print("   → High ACF = clustering (today's vol predicts tomorrow's)")

# 2. Fat tails
print("\n2. FAT TAILS")
kurt = stats.kurtosis(returns)
print(f"   Excess Kurtosis: {kurt:.2f} (Normal = 0)")
print(f"   → Returns have fatter tails than normal!")

Volatility Stylized Facts

1. VOLATILITY CLUSTERING
   Autocorrelation of squared returns (r²):
   Lag 1: 0.083
   Lag 5: 0.092
   Lag 10: 0.032
   → High ACF = clustering (today's vol predicts tomorrow's)

2. FAT TAILS
   Excess Kurtosis: 0.13 (Normal = 0)
   → Returns have fatter tails than normal!


---

## 2. GARCH Model

### Generalized Autoregressive Conditional Heteroskedasticity

Bollerslev (1986): Model time-varying volatility.

### GARCH(1,1) Specification

**Return equation**:
$$r_t = \mu + \epsilon_t$$
$$\epsilon_t = \sigma_t z_t, \quad z_t \sim N(0,1)$$

**Variance equation**:
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

Where:
- $\omega$ > 0: Baseline variance
- $\alpha$ ≥ 0: Impact of recent shock
- $\beta$ ≥ 0: Persistence of past variance
- $\alpha + \beta$ < 1: Stationarity condition

### Unconditional Variance

Long-term average variance:

$$\bar{\sigma}^2 = \frac{\omega}{1 - \alpha - \beta}$$

### Persistence

$\alpha + \beta$ measures how long shocks affect volatility:
- Close to 1: Shocks persist (high persistence)
- Close to 0: Shocks die quickly
- Typical value: 0.90 - 0.99

In [7]:
from arch import arch_model

# Generate data from known GARCH(1,1) process
np.random.seed(42)
n = 2000

true_omega = 0.00001
true_alpha = 0.08
true_beta = 0.90
true_mu = 0.0003

# Simulate
sigma2 = np.zeros(n)
epsilon = np.zeros(n)
returns_sim = np.zeros(n)

sigma2[0] = true_omega / (1 - true_alpha - true_beta)  # Unconditional variance

for t in range(1, n):
    sigma2[t] = true_omega + true_alpha * epsilon[t-1]**2 + true_beta * sigma2[t-1]
    epsilon[t] = np.sqrt(sigma2[t]) * np.random.normal()
    returns_sim[t] = true_mu + epsilon[t]

# Fit GARCH model
garch_model = arch_model(returns_sim * 100, mean='Constant', vol='GARCH', p=1, q=1)
garch_fit = garch_model.fit(disp='off')

print("GARCH(1,1) Model Estimation")
print("="*55)
print(f"\n{'Parameter':<12} | {'True':>10} | {'Estimated':>10}")
print("-"*40)
print(f"{'ω (omega)':<12} | {true_omega*10000:>10.6f} | {garch_fit.params['omega']:>10.6f}")
print(f"{'α (alpha)':<12} | {true_alpha:>10.4f} | {garch_fit.params['alpha[1]']:>10.4f}")
print(f"{'β (beta)':<12} | {true_beta:>10.4f} | {garch_fit.params['beta[1]']:>10.4f}")

persistence = garch_fit.params['alpha[1]'] + garch_fit.params['beta[1]']
print(f"\nPersistence (α + β): {persistence:.4f}")
print(f"Half-life of shock: {np.log(2)/np.log(persistence):.1f} days")

GARCH(1,1) Model Estimation

Parameter    |       True |  Estimated
----------------------------------------
ω (omega)    |   0.100000 |   0.178636
α (alpha)    |     0.0800 |     0.0730
β (beta)     |     0.9000 |     0.8835

Persistence (α + β): 0.9564
Half-life of shock: -15.6 days


### Volatility Forecasting

**1-step ahead forecast**:
$$\sigma_{t+1}^2 = \omega + \alpha \epsilon_t^2 + \beta \sigma_t^2$$

**Multi-step forecast** (h steps ahead):
$$\sigma_{t+h}^2 = \bar{\sigma}^2 + (\alpha + \beta)^{h-1}(\sigma_{t+1}^2 - \bar{\sigma}^2)$$

As h → ∞, forecast converges to unconditional variance.

In [8]:
# Volatility forecasting
forecasts = garch_fit.forecast(horizon=20)
var_forecast = forecasts.variance.iloc[-1].values

# Unconditional variance
uncond_var = garch_fit.params['omega'] / (1 - persistence)

print("GARCH Volatility Forecast")
print("="*50)
print(f"\n{'Horizon':<10} | {'Variance':>12} | {'Annualized Vol':>15}")
print("-"*45)

for h in [1, 5, 10, 20]:
    var = var_forecast[h-1]
    ann_vol = np.sqrt(var * 252) / 100  # Convert back from %
    print(f"{h} day{'s':<5} | {var:>12.6f} | {ann_vol:>14.1%}")

print(f"\nLong-run (unconditional): {np.sqrt(uncond_var * 252)/100:.1%} annualized")
print("\n✓ Forecasts converge to long-run level as horizon increases")

GARCH Volatility Forecast

Horizon    |     Variance |  Annualized Vol
---------------------------------------------
1 days     |     2.888103 |          27.0%
5 days     |     3.085632 |          27.9%
10 days     |     3.287819 |          28.8%
20 days     |     3.579086 |          30.0%

Long-run (unconditional): 32.1% annualized

✓ Forecasts converge to long-run level as horizon increases


---

## 3. EGARCH (Exponential GARCH)

### Motivation

Standard GARCH has limitations:
1. Doesn't capture leverage effect
2. Requires parameter constraints (ω > 0, α ≥ 0, β ≥ 0)

### EGARCH(1,1) Specification

Nelson (1991): Model log variance:

$$\ln(\sigma_t^2) = \omega + \alpha \left( |z_{t-1}| - E|z_{t-1}| \right) + \gamma z_{t-1} + \beta \ln(\sigma_{t-1}^2)$$

Where:
- $z_{t-1} = \epsilon_{t-1}/\sigma_{t-1}$ = standardized residual
- $\gamma$ = leverage parameter (captures asymmetry)

### Leverage Effect

If $\gamma < 0$:
- Negative shock ($z < 0$) → higher volatility
- Positive shock ($z > 0$) → lower volatility
- This is the **leverage effect**!

In [9]:
# Fit EGARCH model with asymmetry parameter (o=1 adds gamma for leverage effect)
egarch_model = arch_model(returns_sim * 100, mean='Constant', vol='EGARCH', p=1, o=1, q=1)
egarch_fit = egarch_model.fit(disp='off')

print("EGARCH(1,1,1) Model")
print("="*50)
print(f"\nParameter estimates:")
print(f"  ω (omega): {egarch_fit.params['omega']:.4f}")
print(f"  α (alpha): {egarch_fit.params['alpha[1]']:.4f}")
print(f"  γ (gamma): {egarch_fit.params['gamma[1]']:.4f}")
print(f"  β (beta):  {egarch_fit.params['beta[1]']:.4f}")

gamma = egarch_fit.params['gamma[1]']
if gamma < 0:
    print(f"\n✓ γ < 0: Leverage effect detected!")
    print(f"  Negative shocks increase volatility more than positive shocks.")
else:
    print(f"\n✗ No leverage effect (γ ≥ 0)")

# Compare log-likelihoods
print(f"\nModel Comparison:")
print(f"  GARCH Log-Likelihood: {garch_fit.loglikelihood:.2f}")
print(f"  EGARCH Log-Likelihood: {egarch_fit.loglikelihood:.2f}")

EGARCH(1,1,1) Model

Parameter estimates:
  ω (omega): 0.0765
  α (alpha): 0.1591
  γ (gamma): -0.0232
  β (beta):  0.9439

✓ γ < 0: Leverage effect detected!
  Negative shocks increase volatility more than positive shocks.

Model Comparison:
  GARCH Log-Likelihood: -4190.16
  EGARCH Log-Likelihood: -4189.28


---

## 4. GJR-GARCH (Asymmetric Effects)

### Motivation

Glosten, Jagannathan, Runkle (1993): Add asymmetric term to standard GARCH.

### GJR-GARCH(1,1) Specification

$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \gamma \epsilon_{t-1}^2 I_{t-1} + \beta \sigma_{t-1}^2$$

Where:
$$I_{t-1} = \begin{cases} 1 & \text{if } \epsilon_{t-1} < 0 \\ 0 & \text{if } \epsilon_{t-1} \geq 0 \end{cases}$$

### Interpretation

Impact of shocks:
- Positive shock: $\alpha$
- Negative shock: $\alpha + \gamma$

If $\gamma > 0$: Negative shocks have greater impact (leverage effect)

In [10]:
# Fit GJR-GARCH model
gjr_model = arch_model(returns_sim * 100, mean='Constant', vol='GARCH', p=1, o=1, q=1)
gjr_fit = gjr_model.fit(disp='off')

print("GJR-GARCH(1,1,1) Model")
print("="*50)
print(f"\nParameter estimates:")
print(f"  ω (omega): {gjr_fit.params['omega']:.6f}")
print(f"  α (alpha): {gjr_fit.params['alpha[1]']:.4f}")
print(f"  γ (gamma): {gjr_fit.params['gamma[1]']:.4f}")
print(f"  β (beta):  {gjr_fit.params['beta[1]']:.4f}")

alpha = gjr_fit.params['alpha[1]']
gamma_gjr = gjr_fit.params['gamma[1]']

print(f"\nImpact of shocks on variance:")
print(f"  Positive shock (good news): α = {alpha:.4f}")
print(f"  Negative shock (bad news):  α + γ = {alpha + gamma_gjr:.4f}")

if gamma_gjr > 0:
    print(f"\n✓ Negative shocks have {(alpha + gamma_gjr)/alpha:.1f}x greater impact!")

GJR-GARCH(1,1,1) Model

Parameter estimates:
  ω (omega): 0.205515
  α (alpha): 0.0605
  γ (gamma): 0.0308
  β (beta):  0.8738

Impact of shocks on variance:
  Positive shock (good news): α = 0.0605
  Negative shock (bad news):  α + γ = 0.0913

✓ Negative shocks have 1.5x greater impact!


### News Impact Curve

The **news impact curve** shows how shocks of different sizes affect volatility:

- GARCH: Symmetric V-shape
- GJR-GARCH: Asymmetric, steeper for negative shocks
- EGARCH: Asymmetric, exponential

In [11]:
# News Impact Curve comparison
shocks = np.linspace(-0.04, 0.04, 100)

# Parameters from fitted models
omega_g = garch_fit.params['omega']
alpha_g = garch_fit.params['alpha[1]']
beta_g = garch_fit.params['beta[1]']

alpha_gjr = gjr_fit.params['alpha[1]']
gamma_gjr = gjr_fit.params['gamma[1]']

# Unconditional variance (baseline)
sigma2_bar = omega_g / (1 - alpha_g - beta_g)

# GARCH: symmetric
garch_impact = omega_g + alpha_g * (shocks * 100)**2 + beta_g * sigma2_bar

# GJR-GARCH: asymmetric
indicator = (shocks < 0).astype(float)
gjr_impact = (gjr_fit.params['omega'] + 
              alpha_gjr * (shocks * 100)**2 + 
              gamma_gjr * (shocks * 100)**2 * indicator +
              gjr_fit.params['beta[1]'] * sigma2_bar)

print("News Impact Curve Analysis")
print("="*50)
print(f"\nImpact of different shock sizes on variance:")
print(f"\n{'Shock':>8} | {'GARCH':>10} | {'GJR-GARCH':>12}")
print("-"*35)

for shock in [-0.03, -0.02, -0.01, 0, 0.01, 0.02, 0.03]:
    idx = np.argmin(np.abs(shocks - shock))
    print(f"{shock*100:>7.1f}% | {garch_impact[idx]:>10.4f} | {gjr_impact[idx]:>12.4f}")

print("\n✓ GJR-GARCH: Negative shocks produce higher variance")

News Impact Curve Analysis

Impact of different shock sizes on variance:

   Shock |      GARCH |    GJR-GARCH
-----------------------------------
   -3.0% |     4.4687 |       4.6247
   -2.0% |     4.0847 |       4.1441
   -1.0% |     3.8732 |       3.8794
    0.0% |     3.7989 |       3.7864
    1.0% |     3.8732 |       3.8480
    2.0% |     4.0847 |       4.0235
    3.0% |     4.4687 |       4.3421

✓ GJR-GARCH: Negative shocks produce higher variance


---

## 5. Multivariate GARCH (DCC)

### Why Multivariate?

For portfolio risk, we need **time-varying correlations** between assets.

### Dynamic Conditional Correlation (DCC)

Engle (2002): Model correlations that change over time.

**Step 1**: Fit univariate GARCH to each asset → standardized residuals $z_t$

**Step 2**: Model correlation dynamics:

$$Q_t = (1 - a - b)\bar{Q} + a(z_{t-1}z'_{t-1}) + bQ_{t-1}$$

$$R_t = \text{diag}(Q_t)^{-1/2} Q_t \text{diag}(Q_t)^{-1/2}$$

Where:
- $Q_t$ = quasi-correlation matrix
- $R_t$ = correlation matrix (proper correlations)
- $\bar{Q}$ = unconditional covariance of $z_t$
- $a, b$ = DCC parameters

### Covariance Matrix

$$H_t = D_t R_t D_t$$

Where $D_t = \text{diag}(\sigma_{1,t}, \sigma_{2,t}, ...)$

In [12]:
# Simplified DCC demonstration
np.random.seed(42)
n = 500

# Simulate two correlated assets with time-varying correlation
# Base correlation changes over time
base_corr = 0.5 + 0.3 * np.sin(2 * np.pi * np.arange(n) / 252)  # Varies 0.2 to 0.8

# Generate correlated returns
asset1 = np.random.normal(0, 0.015, n)
asset2 = base_corr * asset1 + np.sqrt(1 - base_corr**2) * np.random.normal(0, 0.012, n)

# Rolling correlation (simple estimator)
window = 60
rolling_corr = pd.Series(asset1).rolling(window).corr(pd.Series(asset2))

print("Dynamic Correlation Analysis")
print("="*50)
print(f"\nTwo assets with time-varying correlation")
print(f"True correlation range: [{base_corr.min():.2f}, {base_corr.max():.2f}]")
print(f"\nRolling {window}-day correlation:")
print(f"  Min: {rolling_corr.min():.3f}")
print(f"  Max: {rolling_corr.max():.3f}")
print(f"  Mean: {rolling_corr.mean():.3f}")
print(f"\n✓ Correlations change significantly over time!")
print(f"  This matters for portfolio risk management.")

Dynamic Correlation Analysis

Two assets with time-varying correlation
True correlation range: [0.20, 0.80]

Rolling 60-day correlation:
  Min: 0.095
  Max: 0.814
  Mean: 0.553

✓ Correlations change significantly over time!
  This matters for portfolio risk management.


### Applications

**1. Portfolio Risk**:
$$\sigma_p^2(t) = w' H_t w$$

**2. Hedge Ratios**:
$$\beta_t = \frac{h_{12,t}}{h_{22,t}}$$

**3. Correlation Trading**:
- Dispersion trades
- Correlation swaps

In [13]:
# Portfolio risk with time-varying correlation
weights = np.array([0.6, 0.4])

# Calculate portfolio variance at different correlation levels
sigma1, sigma2 = 0.20, 0.15  # Annualized vols

print("Portfolio Risk with Time-Varying Correlation")
print("="*50)
print(f"\nWeights: Asset 1 = {weights[0]:.0%}, Asset 2 = {weights[1]:.0%}")
print(f"Volatilities: σ₁ = {sigma1:.0%}, σ₂ = {sigma2:.0%}")
print(f"\n{'Correlation':>12} | {'Portfolio Vol':>14}")
print("-"*30)

for rho in [0.0, 0.3, 0.5, 0.7, 1.0]:
    # Covariance matrix
    cov_matrix = np.array([
        [sigma1**2, rho * sigma1 * sigma2],
        [rho * sigma1 * sigma2, sigma2**2]
    ])
    
    port_var = weights @ cov_matrix @ weights
    port_vol = np.sqrt(port_var)
    
    print(f"{rho:>12.1f} | {port_vol:>13.1%}")

print("\n✓ Higher correlation → Higher portfolio risk")
print("✓ DCC helps capture these dynamics for better risk estimates")

Portfolio Risk with Time-Varying Correlation

Weights: Asset 1 = 60%, Asset 2 = 40%
Volatilities: σ₁ = 20%, σ₂ = 15%

 Correlation |  Portfolio Vol
------------------------------
         0.0 |         13.4%
         0.3 |         14.9%
         0.5 |         15.9%
         0.7 |         16.8%
         1.0 |         18.0%

✓ Higher correlation → Higher portfolio risk
✓ DCC helps capture these dynamics for better risk estimates


---

## Summary: Week 7 Key Formulas

| Model | Variance Equation |
|-------|------------------|
| GARCH(1,1) | $\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$ |
| EGARCH | $\ln(\sigma_t^2) = \omega + \alpha(|z_{t-1}| - E|z|) + \gamma z_{t-1} + \beta \ln(\sigma_{t-1}^2)$ |
| GJR-GARCH | $\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \gamma \epsilon_{t-1}^2 I_{t-1} + \beta \sigma_{t-1}^2$ |
| DCC | $Q_t = (1-a-b)\bar{Q} + az_{t-1}z'_{t-1} + bQ_{t-1}$ |

### Key Takeaways

1. **Volatility clusters**: Today's vol predicts tomorrow's
2. **GARCH**: Captures clustering and mean reversion
3. **EGARCH/GJR**: Capture leverage effect (asymmetry)
4. **DCC**: Time-varying correlations for portfolio risk
5. **Persistence**: $\alpha + \beta$ measures shock half-life

---

*Next Week: Machine Learning for Trading*